In [1]:
import sqlalchemy as sqla 
from sqlalchemy import create_engine 
import traceback 
import glob 
import os 

import requests 
import time 
import json


from IPython.display import display


import pymysql

In [2]:
import requests
import traceback
import datetime
import time
import os

In [3]:
URI="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="cmcelduff"
PASSWORD="Tullamore1!"

In [4]:
#Connect to database
engine = create_engine("mysql+pymysql://{0}:{1}@{2}:{3}".format(USER, PASSWORD, URI, PORT), echo=True) 
connection = engine.connect()

2023-02-28 09:51:08,835 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-28 09:51:08,856 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:08,963 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-28 09:51:08,965 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:09,357 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-28 09:51:09,359 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
sql = """
CREATE DATABASE IF NOT EXISTS dublin_bikes;
"""
engine.execute(sql)

2023-02-28 09:51:11,125 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dublin_bikes;

2023-02-28 09:51:11,135 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:11,384 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
# command to check database is running
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-28 09:51:11,604 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-28 09:51:11,604 INFO sqlalchemy.engine.Engine [raw sql] {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('awsauthenticationplugin_max_backoff_delay', '2000')
('awsauthenticationplugin_max_retry_count', '3')
('awsauthenticationplugin_retry_delay', '250')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_di

In [7]:
sql_create_schema = "CREATE SCHEMA `dublin_bikes`;"
try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_schema)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-28 09:51:12,144 INFO sqlalchemy.engine.Engine CREATE SCHEMA `dublin_bikes`;
2023-02-28 09:51:12,146 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:12,337 INFO sqlalchemy.engine.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1007, "Can't create database 'dublin_bikes'; database exists")
[SQL: CREATE SCHEMA `dublin_bikes`;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [8]:
# command to create table in mysql
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

    # tom test 
    # tom test 2

2023-02-28 09:51:15,452 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)

2023-02-28 09:51:15,454 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:15,677 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [9]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-28 09:51:18,936 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)

2023-02-28 09:51:18,939 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-28 09:51:19,131 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [10]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations/"
DubBike_API = "7f06972a5ed335cf697379627fd13027274927c7"

def write_to_file(text):
    date_underscores = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.datetime.now())
    with open(r"data\bikes_{}".format(date_underscores).replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        
                                            
def main():
    print(os.path)
    r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
    stations_to_db(r.text)
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
            print(r, now)
            write_to_file(r.text)
            #stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            #if engine is None:
                #pass
        return

if __name__== "__main__":
    main()

<module 'posixpath' from '/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/posixpath.py'>
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 22, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677577749000}
2023-02-28 09:51:38,727 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:51:38,738 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-02-28 09:51:38,863 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 

2023-02-28 09:51:43,262 INFO sqlalchemy.engine.Engine [raw sql] ('Portobello Road', 0, 30, 0, 'dublin', 'PORTOBELLO ROAD', 43, 53.330091, -6.268044, 'OPEN')
2023-02-28 09:51:43,367 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 8, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677577399000}
2023-02-28 09:51:43,573 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:51:43,575 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Street', 0, 20, 0, 'dublin', 'PARNELL STREET', 31, 53.350929, -6.265125, 'OPEN')
2023-02-28 09:51:43,817 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.34

2023-02-28 09:51:48,208 INFO sqlalchemy.engine.Engine [raw sql] ('Earlsfort Terrace', 0, 30, 0, 'dublin', 'EARLSFORT TERRACE', 11, 53.334295, -6.258503, 'OPEN')
2023-02-28 09:51:48,303 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 16, 'available_bikes': 4, 'status': 'OPEN', 'last_update': 1677577794000}
2023-02-28 09:51:48,501 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:51:48,502 INFO sqlalchemy.engine.Engine [raw sql] ('Golden Lane', 0, 20, 0, 'dublin', 'GOLDEN LANE', 17, 53.340803, -6.267732, 'OPEN')
2023-02-28 09:51:48,600 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265},

2023-02-28 09:51:52,909 INFO sqlalchemy.engine.Engine [raw sql] ('Lime Street', 0, 40, 0, 'dublin', 'LIME STREET', 62, 53.346026, -6.243576, 'OPEN')
2023-02-28 09:51:53,005 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 33, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677577728000}
2023-02-28 09:51:53,350 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:51:53,353 INFO sqlalchemy.engine.Engine [raw sql] ('Charlemont Street', 0, 40, 0, 'dublin', 'CHARLEMONT PLACE', 5, 53.330662, -6.260177, 'OPEN')
2023-02-28 09:51:53,448 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': -

2023-02-28 09:51:57,951 INFO sqlalchemy.engine.Engine [raw sql] ("Sir Patrick's Dun", 0, 40, 0, 'dublin', "SIR PATRICK DUN'S", 58, 53.339218, -6.240642, 'OPEN')
2023-02-28 09:51:58,076 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 6, 'available_bikes': 34, 'status': 'OPEN', 'last_update': 1677577801000}
2023-02-28 09:51:58,651 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:51:58,652 INFO sqlalchemy.engine.Engine [raw sql] ('New Central Bank', 0, 40, 0, 'dublin', 'NEW CENTRAL BANK', 66, 53.347122, -6.234749, 'OPEN')
2023-02-28 09:51:58,758 INFO sqlalchemy.engine.Engine COMMIT
{'number': 104, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (CENTRAL)', 'address': 'Grangegorman Lower (Central)', 

...

2023-02-28 09:52:18,187 INFO sqlalchemy.engine.Engine [raw sql] ('St James Hospital (Luas)', 0, 40, 0, 'dublin', 'ST JAMES HOSPITAL (LUAS)', 80, 53.341359, -6.292951, 'OPEN')
2023-02-28 09:52:18,426 INFO sqlalchemy.engine.Engine COMMIT
{'number': 41, 'contract_name': 'dublin', 'name': 'HARCOURT TERRACE', 'address': 'Harcourt Terrace', 'position': {'lat': 53.332763, 'lng': -6.257942}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677577834000}
2023-02-28 09:52:18,637 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 09:52:18,639 INFO sqlalchemy.engine.Engine [raw sql] ('Harcourt Terrace', 0, 20, 0, 'dublin', 'HARCOURT TERRACE', 41, 53.332763, -6.257942, 'OPEN')
2023-02-28 09:52:18,817 INFO sqlalchemy.engine.Engine COMMIT
{'number': 3, 'contract_name': 'dublin', 'name': 'BOLTON STREET', 'address': 'Bolton Street', 'position': {'lat'

2023-02-28 09:52:24,271 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:24,464 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:24,465 INFO sqlalchemy.engine.Engine [raw sql] (108, 0, 35, 1677577496000)
2023-02-28 09:52:24,712 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:24,943 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:24,945 INFO sqlalchemy.engine.Engine [raw sql] (20, 19, 10, 1677577754000)
2023-02-28 09:52:25,107 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:25,304 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:25,306 INFO sqlalchemy.engine.Engine [raw sql] (56, 38, 1, 1677577885000)
2023-02-28 09:52:25,403 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:25,599 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:

2023-02-28 09:52:34,434 INFO sqlalchemy.engine.Engine [raw sql] (99, 27, 2, 1677577649000)
2023-02-28 09:52:34,530 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:34,724 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:34,726 INFO sqlalchemy.engine.Engine [raw sql] (9, 24, 0, 1677577860000)
2023-02-28 09:52:34,821 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:35,073 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:35,075 INFO sqlalchemy.engine.Engine [raw sql] (67, 28, 10, 1677577539000)
2023-02-28 09:52:35,234 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:35,477 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:35,479 INFO sqlalchemy.engine.Engine [raw sql] (116, 1, 29, 1677577620000)
2023-02-28 09:52:35,574 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:35,769 INFO sqlalchemy.eng

2023-02-28 09:52:47,264 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:47,265 INFO sqlalchemy.engine.Engine [raw sql] (71, 3, 37, 1677577845000)
2023-02-28 09:52:47,366 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:47,572 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:47,573 INFO sqlalchemy.engine.Engine [raw sql] (79, 12, 14, 1677577872000)
2023-02-28 09:52:47,787 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:47,986 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:47,988 INFO sqlalchemy.engine.Engine [raw sql] (69, 38, 0, 1677577854000)
2023-02-28 09:52:48,085 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:48,502 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:48,504 INFO sqlalchemy.engine.Engine [raw sql] (25, 14, 16, 16

2023-02-28 09:52:59,479 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:52:59,726 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:52:59,728 INFO sqlalchemy.engine.Engine [raw sql] (109, 15, 14, 1677577828000)
2023-02-28 09:52:59,830 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:00,065 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:00,067 INFO sqlalchemy.engine.Engine [raw sql] (85, 0, 35, 1677577468000)
2023-02-28 09:53:00,168 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:00,487 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:00,489 INFO sqlalchemy.engine.Engine [raw sql] (107, 11, 29, 1677577579000)
2023-02-28 09:53:00,586 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:00,866 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:5

In [11]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = requests.get(STATIONS, params={"contract" : NAME, "apiKey": DubBike_API})

In [12]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 22,
  'available_bikes': 7,
  'status': 'OPEN',
  'last_update': 1677577898000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 18,
  'available_bikes': 2,
  'status': 'OPEN',
  'last_update': 1677577899000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 23,
  'available_bikes': 10,
  'status': 'OPEN',
  'last_update': 1677577759000},


In [13]:
def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        

In [14]:
# just testing out function works here
availability_to_db(r.text)

2023-02-28 09:53:15,699 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:15,700 INFO sqlalchemy.engine.Engine [raw sql] (42, 7, 22, 1677577898000)
2023-02-28 09:53:15,799 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:16,113 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:16,114 INFO sqlalchemy.engine.Engine [raw sql] (30, 2, 18, 1677577899000)
2023-02-28 09:53:16,250 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:16,505 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:16,506 INFO sqlalchemy.engine.Engine [raw sql] (54, 10, 23, 1677577759000)
2023-02-28 09:53:16,602 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:16,797 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:16,798 INFO sqlalchemy.engine.Engine [raw sql] (108, 0, 35, 16

2023-02-28 09:53:27,759 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:27,954 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:27,956 INFO sqlalchemy.engine.Engine [raw sql] (113, 16, 24, 1677577916000)
2023-02-28 09:53:28,145 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:28,343 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:28,345 INFO sqlalchemy.engine.Engine [raw sql] (91, 10, 19, 1677577832000)
2023-02-28 09:53:28,467 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:28,799 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:28,800 INFO sqlalchemy.engine.Engine [raw sql] (99, 27, 2, 1677577649000)
2023-02-28 09:53:28,897 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:29,199 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53

2023-02-28 09:53:39,025 INFO sqlalchemy.engine.Engine [raw sql] (96, 0, 30, 1677577541000)
2023-02-28 09:53:39,197 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:39,404 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:39,406 INFO sqlalchemy.engine.Engine [raw sql] (82, 3, 5, 1677577408000)
2023-02-28 09:53:39,589 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:39,810 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:39,812 INFO sqlalchemy.engine.Engine [raw sql] (76, 3, 35, 1677577941000)
2023-02-28 09:53:39,908 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:40,239 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:40,240 INFO sqlalchemy.engine.Engine [raw sql] (71, 3, 37, 1677577845000)
2023-02-28 09:53:40,359 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:40,631 INFO sqlalchemy.engin

2023-02-28 09:53:50,461 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:50,463 INFO sqlalchemy.engine.Engine [raw sql] (100, 3, 22, 1677577545000)
2023-02-28 09:53:50,557 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:50,819 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:50,821 INFO sqlalchemy.engine.Engine [raw sql] (24, 20, 0, 1677577593000)
2023-02-28 09:53:50,916 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:51,210 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:51,211 INFO sqlalchemy.engine.Engine [raw sql] (64, 24, 16, 1677577892000)
2023-02-28 09:53:51,306 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:51,500 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:51,502 INFO sqlalchemy.engine.Engine [raw sql] (109, 15, 14, 

In [15]:
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

In [16]:
# just testing out function works here
availability_to_db(r.text)

2023-02-28 09:53:58,884 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:58,886 INFO sqlalchemy.engine.Engine [raw sql] (42, 7, 22, 1677577898000)
2023-02-28 09:53:58,981 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:59,210 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:59,212 INFO sqlalchemy.engine.Engine [raw sql] (30, 2, 18, 1677577899000)
2023-02-28 09:53:59,309 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:59,511 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:59,513 INFO sqlalchemy.engine.Engine [raw sql] (54, 10, 23, 1677577759000)
2023-02-28 09:53:59,608 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:53:59,867 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:53:59,869 INFO sqlalchemy.engine.Engine [raw sql] (108, 0, 35, 16

2023-02-28 09:54:09,805 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:54:10,006 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:54:10,007 INFO sqlalchemy.engine.Engine [raw sql] (113, 16, 24, 1677577916000)
2023-02-28 09:54:10,110 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:54:10,464 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:54:10,465 INFO sqlalchemy.engine.Engine [raw sql] (91, 10, 19, 1677577832000)
2023-02-28 09:54:10,560 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:54:10,754 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:54:10,756 INFO sqlalchemy.engine.Engine [raw sql] (99, 27, 2, 1677577649000)
2023-02-28 09:54:10,851 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:54:11,051 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-28 09:54

Exception during reset or similar
Traceback (most recent call last):
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/pymysql/connections.py", line 480, in rollback
    self._read_ok_packet()
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/pymysql/connections.py", line 443, in _read_ok_packet
    pkt = self._read_packet()
  File "/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/pym

KeyboardInterrupt: 

In [ ]:
# just testing out function works here
stations_to_db(r.text)